In [1]:
using Pkg
Pkg.update()
Pkg.instantiate()

using DevCommunities
using SimpleHypergraphs

# g = build_hg("../../hgs/apl.hg")
# num_nodes = nhv(g[1])
# num_edges = nhe(g[1])
# println("Number of nodes: $num_nodes")
# println("Number of edges: $num_edges")

my_tags_so = ["rust" ,"elixir","clojure", "typescript" ,"julia", "python", "delphi", "go" ,"sql", "c#" ,"kotlin", "swift" ,"dart" ,"html", "solidity", "javascript", "f#", "bash" ,"lisp", "apl"]
# order by lexigraphical order
my_tags_so = sort(my_tags_so)

    Updating registry at `~/.julia/registries/General.toml`


  No Changes to `~/Documents/vscode/DevCommunities/Project.toml`
  No Changes to `~/Documents/vscode/DevCommunities/Manifest.toml`


┌ Warning: The plotting functionality of HyperNetX will not work!
│ Conda Python HyperNetX not found.
│ To test your installation try running `using PyCall;pyimport("networkx");pyimport("hypernetx")`
└ @ SimpleHypergraphs ~/.julia/packages/SimpleHypergraphs/lfXYf/src/SimpleHypergraphs.jl:62


┌ Warning: The plotting functionality of HyperNetX will not work!
│ Conda Python HyperNetX not found.
│ To test your installation try running `using PyCall;pyimport("networkx");pyimport("hypernetx")`
└ @ SimpleHypergraphs /Users/ddevin/.julia/packages/SimpleHypergraphs/lfXYf/src/SimpleHypergraphs.jl:62


20-element Vector{String}:
 "apl"
 "bash"
 "c#"
 "clojure"
 "dart"
 "delphi"
 "elixir"
 "f#"
 "go"
 "html"
 "javascript"
 "julia"
 "kotlin"
 "lisp"
 "python"
 "rust"
 "solidity"
 "sql"
 "swift"
 "typescript"

In [ ]:


# for i in 1:length(my_tags_so)
#     g = build_hg("../../hgs/apl.hg")
#     num_nodes = nhv(g[1])
#     num_edges = nhe(g[1])
#     println("Number of nodes: $num_nodes")
#     println("Number of edges: $num_edges")
# end

In [ ]:

# rm_pa = random_model(num_nodes, num_edges)
# average on 10 run | grafo casuale = numero nodes del real graph
# preferential attachament
# hg_save("../../randoms/random500", rm1)
# end
#1000x1000
# motifs_random = [0,87,472,256977,14,112208,0,218303,501,3758514,0,5467523,9213,319352,7407153,144547536,0,32143,0,412031,4,1688973,0,302,112254,1626941]
#100x100
# motifs_random = [0,6,299,2346,11,724,118,5423,30,12916,2,14070,107,1885,20589,86915,0,519,1,3951,4,5240,0,11,695,3849]
#500x500
# motifs_random = [3,19,849,77852,62,17699,35,100549,1168,870515,9,959371,7729,154033,1519553,16308126,0,9921,5,120155,2,226943,0,140,17499,257965]


rm_pa = random_preferential_model(num_nodes, 0.8)
hg_save("../../randoms/randomPIPE", rm_pa)

# open file
r = open("../../randoms/randomPIPE")
# substitue in each line "=true" with ","
run(`sed -i '' 's/=true/,/g' ../../randoms/randomPIPE`)
# remove first line
run(`sed -i '' '1d' ../../randoms/randomPIPE`)
# hash and rewriting id
run(`python3 rewrite.py ../../randoms/randomPIPE $num_edges`)

In [13]:
function build_hg_randoms(path)

    nodes = Dict{Int,Int}()
    degree_nodes = Dict{Int,Int}()
    nodes_per_edge = Dict{Int,Vector{Int}}()

    print(nodes)
    hg = Hypergraph(0,0)
    for line in eachline("../../hgs/apl.hg")
        # split elemnts of the line by comma
        d = split(line, ",")
        vs = map(x -> parse(Int,(strip(replace(x, r"\[|\]" => "")))), d)
        for v in vs
            if !haskey(nodes, v)
                v_id = SimpleHypergraphs.add_vertex!(hg)
                push!(nodes, v=>v_id)
            end
        end
        # add he 
        vertices = Dict{Int, Bool}(nodes[v] => true for v in vs)

        # count degree of each node
        for v in vs
            if !haskey(degree_nodes, v)
                degree_nodes[v] = 0
            end
            degree_nodes[v] += 1
        end
        
        he_id = SimpleHypergraphs.add_hyperedge!(hg; vertices = vertices)
        push!(nodes_per_edge, he_id => vs)  
    end

    sum_degree_nodes = sum(values(degree_nodes))
    sum_degree_edges = sum(values(length(nodes_per_edge)))

    function prob_nodes(v)
        return degree_nodes[v]/sum_degree_nodes
    end

    function prob_edges(e)
        return length(nodes_per_edge[e])/sum_degree_edges
    end

    random_nodes_probs = Dict{Int,Float64}()
    random_edges_probs = Dict{Int,Float64}()

    for key in keys(nodes)
        pr = prob_nodes(key)
        random_nodes_probs[key] = pr
    end

    for key in keys(nodes_per_edge)
        pr = prob_edges(key)
        random_edges_probs[key] = pr
    end

    random_nodes = Dict{Int,Int}()
    random_nodes_per_edge = Dict{Int,Vector{Int}}()

    random_hg = Hypergraph(0,0)

    for i in 1:sum_degree_nodes
        v = sample(collect(keys(sort(random_nodes_probs))), pweights(ProbabilityWeights(collect(values(sort(random_nodes_probs))))))
        e = sample(collect(keys(sort(random_edges_probs))), pweights(ProbabilityWeights(collect(values(sort(random_edges_probs))))))
        if !haskey(random_nodes, v)
            v_id = SimpleHypergraphs.add_vertex!(random_hg)
            push!(random_nodes, v=>v_id)
        end

        if !haskey(random_nodes_per_edge, e)
            random_nodes_per_edge[e] = []
        end
        random_nodes_per_edge[e] = push!(random_nodes_per_edge[e], v)        
    end

    for key in keys(random_nodes_per_edge)
        vertices = Dict{Int, Bool}(random_nodes[v] => true for v in random_nodes_per_edge[key])
        he_id = SimpleHypergraphs.add_hyperedge!(random_hg; vertices = vertices)
    end

        return random_hg
    end

build_hg_randoms (generic function with 1 method)

In [14]:
using StatsBase
rg = build_hg_randoms("../../hgs/apl.hg")



# println(nhv(random_hg))
# println(nhe(random_hg))
# hg_save("../../randoms/realRandom", random_hg)


Dict{Int64, Int64}()

909×958 Hypergraph{Bool, Nothing, Nothing, Dict{Int64, Bool}}:
 nothing  nothing  nothing  nothing  …  nothing  true             nothing
 nothing  nothing  nothing  nothing     nothing      nothing      nothing
 nothing  nothing  nothing  nothing     nothing      nothing  true
 nothing  nothing  nothing  nothing     nothing      nothing      nothing
 nothing  nothing  nothing  nothing     nothing      nothing      nothing
 nothing  nothing  nothing  nothing  …  nothing      nothing      nothing
 nothing  nothing  nothing  nothing     nothing      nothing      nothing
 nothing  nothing  nothing  nothing     nothing      nothing      nothing
 nothing  nothing  nothing  nothing     nothing      nothing      nothing
 nothing  nothing  nothing  nothing     nothing      nothing      nothing
 ⋮                                   ⋱  ⋮                     
 nothing  nothing  nothing  nothing  …  nothing      nothing      nothing
 nothing  nothing  nothing  nothing     nothing      nothing      n

In [ ]:
all_arrays = []
for i in 1:4
    # g = build_hg("../../hgs/apl.hg")
    println("building hg for tag: "*my_tags_so[i])
    g = build_hg("../../hgs/"*my_tags_so[i]*".hg")
    num_nodes = nhv(g[1])
    num_edges = nhe(g[1])
    println("Number of nodes: $num_nodes")
    println("Number of edges: $num_edges")

    total_random_motif_count = [0 for i in 1:26]
    # generate 5 model and do average counting
    for i in range(1,4)
        rm_pa = random_preferential_model(num_nodes, 0.8)
        hg_save("../../randoms/randomPIPE", rm_pa)
        
        # open file
        r1 = open("../../randoms/randomPIPE")
        # substitue in each line "=true" with ","
        run(`sed -i '' 's/=true/,/g' ../../randoms/randomPIPE`)
        # remove first line
        run(`sed -i '' '1d' ../../randoms/randomPIPE`)
        # hash and rewriting id
        run(`python3 rewrite.py ../../randoms/randomPIPE $num_edges`)
        
        run(`bash /Users/ddevin/Documents/MoCHy/run_exact.sh "/Users/ddevin/Documents/vscode/DevCommunities/randoms/randomPIPE.hg"`)
        # run(`bash /Users/ddevin/Documents/MoCHy/run_exact.sh "/Users/ddevin/Documents/vscode/DevCommunities/hgs/apl.hg"`)
        close(r1)
        r2 = open("/Users/ddevin/Documents/vscode/DevCommunities/script_dump/dumpPIPE.txt")
        array = []
        for line in eachline(r2)
            if occursin("motif", line)
                if occursin("counting", line)
                    continue
                end
                # find integers in line after ":"
                motif = parse.(Int64, split(line, ":")[2:end])
                push!(array, motif[1])
            end
        end
        # array of any to array of integers
        array = convert(Array{Int64}, array)
        total_random_motif_count = [x + y for (x, y) in zip(array, total_random_motif_count)]
        close(r2)
        println("done with random model "*string(i))
        flush(stdout)
        # flush
    end

    total_random_motif_count = [x / 5 for x in total_random_motif_count]
    total_random_motif_count = ceil.(Int64, total_random_motif_count)
    println(total_random_motif_count)
    push!(all_arrays, total_random_motif_count)
end

println(all_arrays)
# write to file
open("/Users/ddevin/Documents/vscode/DevCommunities/results/all_motifs_counted.csv", "a") do io
    for i in 1:4
        for j in 1:26
            write(io, string(my_tags_so[i]))
            write(io, string(all_arrays[i][j]))
            write(io, ",")
        end
    end
end





In [ ]:
total_random_motif_count
# convert float64 to integers with ceil of number

In [ ]:
# num_motifs = [1077,818707,23,7,5702,6152,25,1,131,27,10,4,0,0,0,0,5,1,8390,1397,41097,124693,9,582,145,53]
num_motifs = [2070, 11469, 44796591, 58275821, 16910646, 34130003, 16, 93583226, 404737, 293960662, 70840, 175279493, 1, 4172338, 55998189, 362027234, 0, 0, 31323, 12423927, 14751, 27466910, 0, 152556, 716568, 7659269]

function cps(motifs, motifs_random)
    eps = 1 #fixed in the paper
    deltas = []
    sum_deltas = 0
    for i in 1:length(motifs)
        t = (motifs[i]-motifs_random[i])/(motifs[i]+motifs_random[i]+eps)
        push!(deltas, t)
        sum_deltas += t^2
    end
    cp = []
    for i in 1:length(motifs)
        push!(cp, deltas[i]/sqrt(sum_deltas))
    end
    return cp
end

# cp = cps(num_motifs, motifs_random)
# println(cp)

In [ ]:
using CSV, DataFrames
data = CSV.read("../../hgs/motifs.csv", DataFrame, header=false)

# for each column create an array of the values
w = open("../../results/cpprofilesAPL.csv", "w")

for i in 1:size(data, 2)
    col = data[:, i]
    col = convert(Array{Int64, 1}, col)
    cp = cps(col, num_motifs)
    new_line = join(cp, ",")
    
    write(w, new_line * "\n")
end
close(w)



In [ ]:
using Plots, LinearAlgebra
using DelimitedFiles
# read matrix from csv file
m = readdlm("../../results/cpprofiles1000.csv", ',', Float64)
h = heatmap(m)

In [ ]:
using Plots, LinearAlgebra
using DelimitedFiles
# read matrix from csv file
m = readdlm("../../results/cpprofiles100.csv", ',', Float64)
heatmap(m, yflip=true)

In [ ]:
# read matrix from csv file
m = readdlm("../../results/cpprofiles500.csv", ',', Float64)
heatmap(m, yflip=true)

In [ ]:
ccs = SimpleHypergraphs.get_connected_components(g[1])
ccs = sort(ccs, by=length, rev=true)
# distribution of component sizes
ccs_len = length.(ccs)



In [ ]:
using PyPlot
dict_degree = DevCommunities.degree_histogram(g[1], normalized=true)
# bar plot of the degree distribution
fig,axs = subplots()
axs.bar(collect(keys(dict_degree)), collect(values(dict_degree)), color="red")
gcf()
